In [1]:
# les bibliotheques nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet


## Sommaire
1. Import
2. Préparation
3. Baseline
4. Modèles linéaires
5. Modèles d’arbres
6. Méthodes d’ensemble
7. Conclusion

CSV Nettoyé

In [2]:
print("\n1. CHARGEMENT ET PRÉPARATION DES DONNÉES")
df_work = pd.read_csv("clean_seattle.csv")
print(f"Shape des données : {df_work.shape}")
df_work.head()



1. CHARGEMENT ET PRÉPARATION DES DONNÉES
Shape des données : (689, 48)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,log_TotalGHGEmissions,log_SiteEnergyUse_kBtu,BuildingAge,AvgFloorArea,GFATotal_per_Building
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,659000030,...,False,Compliant,Unknown,249.98,2.83,5.525373,15.793246,89,7369.500000,88434.0
1,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,659000475,...,False,Compliant,Unknown,2089.28,2.19,7.645053,18.100297,47,23319.756098,956110.0
2,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,659000640,...,False,Compliant,Unknown,286.43,4.67,5.660979,15.731637,90,6132.000000,61320.0
3,9,2016,Nonresidential COS,Other,West Precinct,810 Virginia St,Seattle,WA,98101.0,660000560,...,False,Compliant,Unknown,301.81,3.10,5.713106,16.307609,17,48644.000000,97288.0
4,10,2016,NonResidential,Hotel,Camlin,1619 9th Avenue,Seattle,WA,98101.0,660000825,...,False,Compliant,Unknown,176.14,2.12,5.176940,15.566239,90,7546.181818,83008.0


Variables

In [3]:
num_cols = [
    'PropertyGFATotal', 'AvgFloorArea',
    'NumberofFloors', 'NumberofBuildings', 
    'BuildingAge', 'GFATotal_per_Building',
    'Latitude', 'Longitude'
]
#  pour le moment on ne prend pas 'ENERGYSTARScore'
cat_cols = ['PrimaryPropertyType']
target = 'log_TotalGHGEmissions'
print(f"✓ Variables numériques : {len(num_cols)}")
print(f"✓ Variables catégorielles : {len(cat_cols)}")
print(f"✓ Variable cible : {target}")

# Préprocesseur - normaliser les données avant entrainement
preproc = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols)
])

✓ Variables numériques : 8
✓ Variables catégorielles : 1
✓ Variable cible : log_TotalGHGEmissions


Mise en place de la validation croisée

In [4]:
# configuration de la validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Liste des métriques 
scoring = {
    'MAE':  'neg_mean_absolute_error',
    'RMSE': 'neg_root_mean_squared_error', 
    'R2':   'r2'
}

Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone,
elle est exprimée dans la même unité que la cible
et enfin elle complète la mesure du R² en donnant une mesure concrète de l erreur moyenne attendue 


Préparation des données

In [6]:

# Sélection X, y - X étant les variables retenues et y la cible
X = df_work[num_cols + cat_cols].copy()
y = df_work[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"✓ Split réalisé : {len(X_train)} train, {len(X_test)} test")

✓ Split réalisé : 551 train, 138 test


BASELINE

In [7]:
# mise en place du modèle de base DummyRegressor - #prédit juste la moyenne
print("1. MODÈLE BASELINE (DummyRegressor)")
print("-"*50)

dummy_pipe = Pipeline([
    ('prep', preproc),
    ('model', DummyRegressor(strategy='mean'))
])

# Cross-validation du baseline
cv_dummy = cross_validate(dummy_pipe, X_train, y_train, cv=cv, scoring=scoring)

baseline_r2 = cv_dummy['test_R2'].mean()
baseline_rmse = -cv_dummy['test_RMSE'].mean()
baseline_mae = -cv_dummy['test_MAE'].mean()

print(f"Baseline (DummyRegressor - moyenne) :")
print(f"  R2 CV    : {baseline_r2:.4f} ± {cv_dummy['test_R2'].std():.4f}")
print(f"  RMSE CV  : {baseline_rmse:.4f} ± {cv_dummy['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {baseline_mae:.4f} ± {cv_dummy['test_MAE'].std():.4f}")
print(f" on note ici que nos modèles à entrainer doivent avoir des résultats inférieurs à ces moyennes de base") 

1. MODÈLE BASELINE (DummyRegressor)
--------------------------------------------------
Baseline (DummyRegressor - moyenne) :
  R2 CV    : -0.0066 ± 0.0062
  RMSE CV  : 1.4870 ± 0.1222
  MAE CV   : 1.1643 ± 0.1213
 on note ici que nos modèles à entrainer doivent avoir des résultats inférieurs à ces moyennes de base


In [8]:
print("\n2. RÉGRESSION LINÉAIRE SIMPLE")
print("-" * 50)

# La régression linéaire sert de baseline pour comprendre
# les relations linéaires dans nos données avant d'ajouter de la régularisation
linear_pipe = Pipeline([('prep', preproc), ('model', LinearRegression())])
cv_linear  = cross_validate(
    linear_pipe,
    X_train, y_train,
    cv=cv,
    scoring=scoring,
    return_train_score=True   # ← on récupère train_X et test_X
)
#ici on note une petite amélioration par rapport au baseline, on explique 23% de la variance ce qui est mieux et les métriques RMSE/MAE sont plus élévés en test qu'en entrainement mais
#de manière modérée -voir ensuite avec des modèles de regularisation (ridge/lasso) et des méthodes non linéaires pour voir une amélioration de l'écart type
print("Régression Linéaire (CV interne)")
print(f"  R^2 train CV : {cv_linear ['train_R2'].mean():.4f} ± {cv_linear ['train_R2'].std():.4f}")
print(f"  R^2  test  CV : {cv_linear ['test_R2'].mean():.4f} ± {cv_linear ['test_R2'].std():.4f}")
print(f"  RMSE train CV : {-cv_linear ['train_RMSE'].mean():.4f} ± {cv_linear ['train_RMSE'].std():.4f}")
print(f"  RMSE test  CV : {-cv_linear ['test_RMSE'].mean():.4f} ± {cv_linear ['test_RMSE'].std():.4f}")
print(f"  MAE train CV  : {-cv_linear ['train_MAE'].mean():.4f} ± {cv_linear ['train_MAE'].std():.4f}")
print(f"  MAE test  CV  : {-cv_linear ['test_MAE'].mean():.4f} ± {cv_linear ['test_MAE'].std():.4f}")


2. RÉGRESSION LINÉAIRE SIMPLE
--------------------------------------------------
Régression Linéaire (CV interne)
  R^2 train CV : 0.4993 ± 0.0180
  R^2  test  CV : 0.2266 ± 0.2691
  RMSE train CV : 1.0541 ± 0.0269
  RMSE test  CV : 1.2752 ± 0.1674
  MAE train CV  : 0.8251 ± 0.0232
  MAE test  CV  : 0.9305 ± 0.0767


In [ ]:

print("3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)")
print("-"*50)

#  La régularisation aide à :
# - Ridge : réduire l'overfitting en pénalisant les coefficients élevés
# - Lasso : sélection de variables en annulant certains coefficients
# - ElasticNet : combinaison des avantages de Ridge et Lasso
# modèles linéaires

# Pipeline de base
reg_pipe = Pipeline([
    ('prep', preproc),
    ('model', Ridge())  # Placeholder qui sera remplacé par GridSearchCV
])

# Grille de paramètres - pour tester plusieurs combinaisons d'hyperparamètres pour donner le meilleur compromis biais/variance 
#alpha = force de pénalité pour essayer de réduire la variance (moins de sur-entrainement), stabiliser les poids de chaque feature, et sélectionner des variables via Lasso si nécessaire
#Rideg pénalise la somme dess poids au carré, plus Alpha est grand plus les coeff sont retrécies vers 0, Lasso pénalise la somme des valeurs absolues et Elastic net combien les 2 (sélection et stabilisation)
param_grid = [
    {
        'model': [Ridge()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0] # échelle logarithmique pour balayer plusieurs ordres de grandeur et voir si le modèle préfère une pénalité nulle, modérée ou forte
    },
    {
        'model': [Lasso()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]  #idem
    },
    {
        'model': [ElasticNet()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
        'model__l1_ratio': [0.1, 0.5, 0.7, 0.9] # curseur de mélange pour "doser" Lasso et Ridge - exemple 0.5 c'est 50% de Lasso et 50% de Ridge , 0.9 surtout Lasso et peu de Ridge
    }
]

print("Optimisation des hyperparamètres par GridSearchCV...")
gs_reg = GridSearchCV(
    reg_pipe,
    param_grid,
    cv=cv,
    scoring=scoring,
    refit='RMSE', #moins standart que R^2 mais logique dans ma démarche
    n_jobs=-1,
    return_train_score=True  # important pour avoir mean_train_...
)
gs_reg.fit(X_train, y_train) #on fit le jeu de données

cv_res = gs_reg.cv_results_
best_idx = gs_reg.best_index_                # l’indice de la meilleure config trouvée

best_reg_r2   = cv_res['mean_test_R2'][best_idx]
best_reg_rmse = -cv_res['mean_test_RMSE'][best_idx]
best_reg_mae  = -cv_res['mean_test_MAE'][best_idx]

# On récupère les meilleurs index par type
best_ridge_idx = None
best_lasso_idx = None
best_enet_idx = None

for i, p in enumerate(cv_res['params']):
    if isinstance(p['model'], Ridge):
        if best_ridge_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_ridge_idx]:
            best_ridge_idx = i
    elif isinstance(p['model'], Lasso):
        if best_lasso_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_lasso_idx]:
            best_lasso_idx = i
    elif p['model'].__class__.__name__ == 'ElasticNet':
        if best_enet_idx is None or cv_res['mean_test_R2'][i] > cv_res['mean_test_R2'][best_enet_idx]:
            best_enet_idx = i

print("\n=== Résultats Régularisation ===")

for name, idx in [('Ridge', best_ridge_idx), ('Lasso', best_lasso_idx), ('ElasticNet', best_enet_idx)]:
    if idx is None:
        print(f"{name} : Non testé")
        continue
   
    print(f"\n{name} :")
    print(f"  Meilleurs paramètres : {cv_res['params'][idx]}")
    print(f"  R² train CV : {cv_res['mean_train_R2'][idx]:.4f}")
    print(f"  R² test  CV : {cv_res['mean_test_R2'][idx]:.4f}")
    print(f"  RMSE train CV : {-cv_res['mean_train_RMSE'][idx]:.4f}")
    print(f"  RMSE test  CV : {-cv_res['mean_test_RMSE'][idx]:.4f}")
    print(f"  MAE train CV  : {-cv_res['mean_train_MAE'][idx]:.4f}")
    print(f"  MAE test  CV  : {-cv_res['mean_test_MAE'][idx]:.4f}")
## Les trois régularisations apportent un petit gain de robustesse par rapport à la régression simple,
# mais le R² test reste limité (~0.25), signe que d'autres modèles (Boosting, Stacking) sont nécessaires 
# pour améliorer l'entrainement.
   

3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)
--------------------------------------------------
Optimisation des hyperparamètres par GridSearchCV...



=== Résultats Régularisation ===

Ridge :
  Meilleurs paramètres : {'model': Ridge(), 'model__alpha': 10.0}
  R² train CV : 0.4417
  R² test  CV : 0.2556
  RMSE train CV : 1.1132
  RMSE test  CV : 1.2613
  MAE train CV  : 0.8790
  MAE test  CV  : 0.9347

Lasso :
  Meilleurs paramètres : {'model': Lasso(), 'model__alpha': 0.01}
  R² train CV : 0.4485
  R² test  CV : 0.2695
  RMSE train CV : 1.1063
  RMSE test  CV : 1.2531
  MAE train CV  : 0.8705
  MAE test  CV  : 0.9324

ElasticNet :
  Meilleurs paramètres : {'model': ElasticNet(), 'model__alpha': 0.01, 'model__l1_ratio': 0.9}
  R² train CV : 0.4500
  R² test  CV : 0.2673
  RMSE train CV : 1.1048
  RMSE test  CV : 1.2542
  MAE train CV  : 0.8692
  MAE test  CV  : 0.9322


In [ ]:
print("4. Première exploration des modèles non-linéaires (GradientBoosting et Random Forest)")
print("-"*50)
# modèles d'arbres décisionnels - non linéaires - POUR VOIR SI ILS SURPASSENT LES MODELES LINEAIRES
# --- Pipeline pour le Gradient Boosting ---
boost_pipe = Pipeline([
    ('prep', preproc),
    ('model', GradientBoostingRegressor(random_state=42))
])

# --- Grille d’hyper-paramètres (préfixe 'model__') ---
param_grid_boost = {
    'model__n_estimators':  [100, 200, 300], # nombre d'arbres déterminés, plus il y a d'arbres plus le modèle est capable de corriger les erreurs résiduelles mais plus long à entraîner
    'model__learning_rate': [0.01, 0.1, 0.2], # taux apprentissage - plus il est grand plus le modèle apprend vite mais risque de sur-apprentissage - 0.1 valeur par défaut - 0.01 plus lent mais plus stable et 0.2 plus rapide
    'model__max_depth':     [3, 5, 7] # profondeur des arbres - 3 classique, 5 plus de flexibilité - 7 test si plus profond si il est risqué - AMELIORER LE COMMENTAIRE 
}

# --- Lancement de la GridSearch ---
gs_boost = GridSearchCV(
    boost_pipe,
    param_grid_boost,
    cv=cv,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1,
    refit='RMSE'
)
gs_boost.fit(X_train, y_train)
print("→ Meilleurs params GradientBoost :", gs_boost.best_params_)

# Pipeline pour Random Forest
rf_pipe = Pipeline([
    ('prep', preproc),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Grille de recherche (attention au préfixe model__)
param_grid_rf = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth':    [None, 5, 10]
}

# Lancement de la GridSearch
gs_rf = GridSearchCV(
    rf_pipe,
    param_grid_rf,
    cv=cv,
    scoring=scoring,
    n_jobs=-1,
    refit='RMSE',
    return_train_score=True
)
gs_rf.fit(X_train, y_train)
print("→ Meilleurs params RF :", gs_rf.best_params_)

# Récupère tous les scores CV pour Gradient Boosting
cv_boost = gs_boost.cv_results_
best_boost_idx = gs_boost.best_index_

print("\n=== Résultats Gradient Boosting ===")
print(f"R² train CV : {cv_boost['mean_train_R2'][best_boost_idx]:.4f}")
print(f"R² test  CV : {cv_boost['mean_test_R2'][best_boost_idx]:.4f}")
print(f"RMSE train CV : {-cv_boost['mean_train_RMSE'][best_boost_idx]:.4f}")
print(f"RMSE test  CV : {-cv_boost['mean_test_RMSE'][best_boost_idx]:.4f}")
print(f"MAE train CV : {-cv_boost['mean_train_MAE'][best_boost_idx]:.4f}")
print(f"MAE test  CV : {-cv_boost['mean_test_MAE'][best_boost_idx]:.4f}")

# Idem pour Random Forest
cv_rf = gs_rf.cv_results_
best_rf_idx = gs_rf.best_index_

print("\n=== Résultats Random Forest ===")
print(f"R² train CV : {cv_rf['mean_train_R2'][best_rf_idx]:.4f}")
print(f"R² test  CV : {cv_rf['mean_test_R2'][best_rf_idx]:.4f}")
print(f"RMSE train CV : {-cv_rf['mean_train_RMSE'][best_rf_idx]:.4f}")
print(f"RMSE test  CV : {-cv_rf['mean_test_RMSE'][best_rf_idx]:.4f}")
print(f"MAE train CV : {-cv_rf['mean_train_MAE'][best_rf_idx]:.4f}")
print(f"MAE test  CV : {-cv_rf['mean_test_MAE'][best_rf_idx]:.4f}")

# interpretation des résultats
# pour Gradient Boosting, on voit que le R2 modèle apprends bien sur le train, que le r2 test géréralise mieux que les modèles précédents (0.45 / 025 (reg lineaire-0.27 ridge/lasso/elasticNet)
#l'écart train/test est correct pour un arbre de décision : 0.77 à 0.45 - peu de sur apprentissage - bon compromis biais/variance, RMSE test plus faible que le linéaire
# pour Random Forest, R² train CV ≈ 0.85 → très bon fit sur le train (modèle flexible) et  R² test CV = 0.42 : très proche du Gradient Boosting, mais l’écart train/test est plus large (0.85 → 0.42).
# Décalage indiuqant que RF est plus sur du sur-apprentissage qu'un GB réglé finement, RMSE test ≈ 1.13 : un peu plus haut que Gradient Boost, donc globalement il performe un peu moins bien.

4. Première exploration des modèles non-linéaires (GradientBoosting et Random Forest)
--------------------------------------------------
→ Meilleurs params GradientBoost : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100}
→ Meilleurs params RF : {'model__max_depth': 10, 'model__n_estimators': 300}

=== Résultats Gradient Boosting ===
R² train CV : 0.7684
R² test  CV : 0.4488
RMSE train CV : 0.7170
RMSE test  CV : 1.1000
MAE train CV : 0.5687
MAE test  CV : 0.8856

=== Résultats Random Forest ===
R² train CV : 0.8521
R² test  CV : 0.4203
RMSE train CV : 0.5729
RMSE test  CV : 1.1297
MAE train CV : 0.4670
MAE test  CV : 0.9065


Méthodes d'ensemble : Bagging / Boosting / Stacking

In [11]:

print("\n5. MÉTHODES D'ENSEMBLE")
print("-" * 50)
# Techniques pour combiner plusieurs modèles
# A) BAGGING - Réduit la variance en combinant plusieurs modèles entraînés sur
# différents échantillons des données d'entraînement
print("A) BAGGING (Bootstrap Aggregating)")
bagging_pipe = Pipeline([
    ('prep', preproc),
    ('model', BaggingRegressor(
        n_estimators=50,
        random_state=42,
        n_jobs=-1
    ))
])
cv_bagging = cross_validate(bagging_pipe, X_train, y_train,cv=cv, scoring=scoring)

best_boost = gs_boost.best_estimator_
best_boost.fit(X_train, y_train)
y_pred_train_boost = best_boost.predict(X_train)
y_pred_test_boost  = best_boost.predict(X_test)
print(f"R^2 train : {r2_score(y_train, y_pred_train_boost):.4f}") # gain modeste par rapport aux linéaires (≃0.25), mais écart train/test ≃0.26  
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_boost):.4f}") # montre un sur‐apprentissage toujours présent.  
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_boost)):.4f}")
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_boost)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_boost):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_boost):.4f}")

# B) BOOSTING
print("\nB) BOOSTING (Gradient Boosting)")
# Justification : Réduit le biais en combinant séquentiellement des modèles faibles,
# chaque nouveau modèle corrigeant les erreurs du précédent

boosting_pipe = Pipeline([
    ('prep', preproc),
    ('model', HistGradientBoostingRegressor(random_state=42))
])

# Optimisation des hyperparamètres pour le boosting
param_grid_boost = {
    'model__max_iter': [100, 200],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5]
}

gs_boost = GridSearchCV(boosting_pipe, param_grid_boost, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
gs_boost.fit(X_train, y_train)

cv_boost_results = cross_validate(gs_boost.best_estimator_, X_train, y_train, cv=cv, scoring=scoring)

best_boost = gs_boost.best_estimator_
best_boost.fit(X_train, y_train)
y_pred_train_boost = best_boost.predict(X_train)
y_pred_test_boost  = best_boost.predict(X_test)
print(f"R^2 train : {r2_score(y_train, y_pred_train_boost):.4f}") #étonnant - même résultat que bagging 
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_boost):.4f}")
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_boost)):.4f}")
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_boost)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_boost):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_boost):.4f}")

# C) STACKING
print("\nC) STACKING")
# Justification : Combine les prédictions de plusieurs modèles via un meta-learner
# qui apprend comment optimiser la combinaison

# Modèles de base pour le stacking
base_models = [
    ('ridge', gs_reg.best_estimator_['model']),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('gb', gs_boost.best_estimator_['model'])
]

stacking_pipe = Pipeline([
    ('prep', preproc),
    ('model', StackingRegressor(
        estimators=base_models,
        final_estimator=LinearRegression(),
        cv=3,  # CV interne pour éviter l'overfitting
        n_jobs=-1
    ))
])

cv_stacking = cross_validate(stacking_pipe, X_train, y_train, cv=cv, scoring=scoring)
stacking_pipe.fit(X_train, y_train)
y_pred_train_stacking = stacking_pipe.predict(X_train)
y_pred_test_stacking  = stacking_pipe.predict(X_test)
print(f"R^2 train : {r2_score(y_train, y_pred_train_stacking):.4f}") # +0.08 de R2 test par rapport à Bagging/Boosting
print(f"R^2 test  : {r2_score(y_test,  y_pred_test_stacking):.4f}")  #avec un écart train/test réduit (0.33 vs 0.26)   
print(f"RMSE train : {np.sqrt(mean_squared_error(y_train, y_pred_train_stacking)):.4f}")# meilleure maîtrise du sur‐apprentissage.
print(f"RMSE test  : {np.sqrt(mean_squared_error(y_test,  y_pred_test_stacking)):.4f}")
print(f"MAE train  : {mean_absolute_error(y_train, y_pred_train_stacking):.4f}")
print(f"MAE test   : {mean_absolute_error(y_test,  y_pred_test_stacking):.4f}")

# Bagging est un bon compromis, a une bonne capacité à lisser la variance.
# Boosting : un peu trop sous-appris, pourrait être mieux tuné (plus d’arbres, learning_rate ajusté).
# Stacking : améliore Boosting mais reste un peu plus lourd et donc ne surpasse pas Bagging.



5. MÉTHODES D'ENSEMBLE
--------------------------------------------------
A) BAGGING (Bootstrap Aggregating)
R^2 train : 0.7342
R^2 test  : 0.4387
RMSE train : 0.7684
RMSE test  : 1.0262
MAE train  : 0.6070
MAE test   : 0.7663

B) BOOSTING (Gradient Boosting)
R^2 train : 0.5799
R^2 test  : 0.3207
RMSE train : 0.9661
RMSE test  : 1.1289
MAE train  : 0.7697
MAE test   : 0.8193

C) STACKING
R^2 train : 0.7293
R^2 test  : 0.3995
RMSE train : 0.7754
RMSE test  : 1.0615
MAE train  : 0.6155
MAE test   : 0.7871


In [13]:
print('Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone,\nelle est exprimée dans la même unité que la cible\net enfin elle complète la mesure du R² en donnant une mesure concrète de l erreur moyenne attendue ')

Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone,
elle est exprimée dans la même unité que la cible
et enfin elle complète la mesure du R² en donnant une mesure concrète de l erreur moyenne attendue 


Comparaison des modèles

In [12]:
print("\n6. COMPARAISON DES MODÈLES (Validation Croisée)")
print("-" * 70)
# evaluer la meilleure performance de modèles
# 1) On rassemble les scores R2, RMSE, MAE calculés en cross-validation pour chaque modèle testé
results_cv = {
    'Baseline (Dummy)': (
        baseline_r2,
        baseline_rmse,
        baseline_mae
    ),
    'Régression Linéaire': (
        cv_linear ['test_R2'].mean(),
        -cv_linear ['test_RMSE'].mean(),
        -cv_linear ['test_MAE'].mean()
    ),
    
    'Meilleur Régularisé': (
        best_reg_r2,      
        best_reg_rmse,
        best_reg_mae
    ),
    'Bagging': (
        cv_bagging['test_R2'].mean(),
        -cv_bagging['test_RMSE'].mean(),
        -cv_bagging['test_MAE'].mean()
    ),
    'Boosting': (
        cv_boost_results['test_R2'].mean(),
        -cv_boost_results['test_RMSE'].mean(),
        -cv_boost_results['test_MAE'].mean()
    ),
    'Stacking': (
        cv_stacking['test_R2'].mean(),
        -cv_stacking['test_RMSE'].mean(),
        -cv_stacking['test_MAE'].mean()
    )
}
# Affichage formaté des résultats CV
print(f"{'Modèle':<20} {'R^2_CV':<8} {'RMSE_CV':<8} {'MAE_CV':<8}")
print("-" * 50)
for name, (r2, rmse, mae) in results_cv.items():
    print(f"{name:<20} {r2:<8.4f} {rmse:<8.4f} {mae:<8.4f}")
# Sélection du meilleur modèle en fonction du plus faible RMSE
best_model_name = min(results_cv.keys(), key=lambda x: results_cv[x][1])
print(f"\nMeilleur modèle (RMSE le plus faible) : {best_model_name}")

# 2) Évaluation finale (train vs test)
print("\n6b. ÉVALUATION FINALE TRAIN / TEST")
print("-" * 70)
# Ici on refait une évaluation sur train / test pour comparer : R2, RMSE et MAE sur train et test
# On calcule aussi : l'amélioration en RMSE par rapport au Dummy sur le test
# et La cohérence entre RMSE CV et RMSE test
print(f"{'Modèle':<20} {'R^2 tr/te':<15} {'RMSE tr/te':<15} {'MAE tr/te':<15}")
print("-" * 70)
# Map des modèles entraînés avec les meilleurs hyperparamètres
models_map = {
    'Baseline (Dummy)': dummy_pipe,
    'Régression Linéaire': linear_pipe,
    'Meilleur Régularisé': gs_reg.best_estimator_,
    'Bagging': bagging_pipe,
    'Boosting': gs_boost.best_estimator_,
    'Stacking': stacking_pipe
}

for name, mdl in models_map.items():
    # Réentraîner sur train complet
    mdl.fit(X_train, y_train)
    y_tr = mdl.predict(X_train)
    y_te = mdl.predict(X_test)
     # Scores train/test
    r2_tr = r2_score(y_train, y_tr)
    r2_te = r2_score(y_test,  y_te)
    rmse_tr = np.sqrt(mean_squared_error(y_train, y_tr))
    rmse_te = np.sqrt(mean_squared_error(y_test,  y_te))
    mae_tr  = mean_absolute_error(y_train, y_tr)
    mae_te  = mean_absolute_error(y_test,  y_te)
    # amélioration vs baseline (Dummy)
    dummy_rmse = np.sqrt(mean_squared_error(y_test, dummy_pipe.predict(X_test)))
    improvement = (dummy_rmse - rmse_te) / dummy_rmse * 100
    # cohérence CV/Test RMSE
    cv_rmse = results_cv[name][1]
    coherence = abs(cv_rmse - rmse_te) / cv_rmse * 100
    #affichage
    print(f"{name:<20} {r2_tr:.3f}/{r2_te:.3f}     "
          f"{rmse_tr:.3f}/{rmse_te:.3f}      "
          f"{mae_tr:.3f}/{mae_te:.3f}      "
          f"{improvement:+.1f}%       "
          f"{coherence:.1f}%")
    #Gain net par rapport au Dummy valide l'intérêt du modèle.e, mais CV vs test assez inégal (16 %), signe d’un peu de variance



6. COMPARAISON DES MODÈLES (Validation Croisée)
----------------------------------------------------------------------
Modèle               R^2_CV   RMSE_CV  MAE_CV  
--------------------------------------------------
Baseline (Dummy)     -0.0066  1.4870   1.1643  
Régression Linéaire  0.2266   1.2752   0.9305  
Meilleur Régularisé  0.2655   1.2515   0.9236  
Bagging              0.4194   1.1302   0.9028  
Boosting             0.3935   1.1543   0.9277  
Stacking             0.4125   1.1320   0.8862  

Meilleur modèle (RMSE le plus faible) : Bagging

6b. ÉVALUATION FINALE TRAIN / TEST
----------------------------------------------------------------------
Modèle               R^2 tr/te       RMSE tr/te      MAE tr/te      
----------------------------------------------------------------------
Baseline (Dummy)     0.000/-0.007     1.491/1.375      1.163/1.097      +0.0%       7.5%
Régression Linéaire  0.487/0.392     1.067/1.068      0.833/0.817      +22.3%       16.2%
Meilleur Régularis

 7. CONCLUSION ET JUSTIFICATIONS TECHNIQUES
 

    **Métrique choisie : RMSE**  
    Pénalise fortement les grosses erreurs, critique pour la prédiction d’émissions de carbone où les écarts importants sont coûteux.
    Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone, elle est exprimée dans la même unité que la cible et enfin elle complète la mesure du R^2 en donnant une mesure concrète de l erreur moyenne attendue 

    **Modèle final sélectionné : BaggingRegressor**  
        - **RMSE CV : 1.130** (meilleur de tous)  
        - **Écart R^2 train→test : 0.92→0.34** (sur-apprentissage modéré)  
        - **Cohérence CV/Test : 1.8 %** (le plus faible, gage de fiabilité)  
        - **Amélioration vs baseline : +19.3 %**  
    le Bagging a montré la meilleure cohérence entre son score de validation croisée et son score de test (1.8% d'écart). Cette stabilité me semble plus importante qu'un gain de performance minime, car elle suggère que le modèle est plus fiable et généralisera mieux à de nouvelles données
    
    Bagging offre donc le **meilleur compromis biais/variance** :  
        1. **Performance** au top en validation croisée  
        2. **Stabilité** d’une exécution à l’autre (faible variabilité)  
        3. **Rapidité** de calcul et simplicité de tuning  
        4. **Robustesse** aux données bruitées  


